# Exercise 4

Group Members: Luis Pazos Clemens, Robert Freund, Eugen Dizer

Deadline: 22.12.2020, 16:00.

# Preliminaries

In [1]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
%matplotlib inline

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [3]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                features_new = np.random.permutation(valid_features)[:D_try] # randomly shuffel and only take first D_try
                new_nodes = make_density_split_node(node, n, features_new)
                stack += [new_nodes[0], new_nodes[1]]
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, n)

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        if np.all(np.logical_and(self.box[0] <= x, x <= self.box[1])):
            return self.prior * leaf.response
        else:
            return 0

In [4]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box
    v = np.prod(M-m)
    if v <= 0:
        raise RuntimeError("zero volume (should not happen)")

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary:
        # Otherwise we would get thresholds which lie at the position of data points.
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = (data_unique[:-1] + data_unique[1:]) / 2
        
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # compute the number of instances in left and right node 
            nl = np.sum(node.data[:,j] <= t) 
            nr = n - nl
            # compute volumes of left and right nodes
            vl = t / (M[j] - m[j])   # vl = v * t / (M[j] - m[j])
            vr = 1.0 - vl            # vr = v - vl 
            # Notice actual volumes would be the commented lines which differ by the constant factor v. 
            # We are using the more efficient computation because it will not make any 
            # difference in comparing the looErr's for different thresholds.
            if vl == 0 or vr == 0:
                continue
            # compute looErr's 
            el = nl / N / vl * (nl / N - 2.0 * (nl-1) / (N-1))
            er = nr / N / vr * (nr / N - 2.0 * (nr-1) / (N-1))
            # choose the best threshold that minimizes sum of looErr
            loo_error = el + er
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min, :]
    left.box = m.copy(), M.copy()
    left.box[1][j_min] = t_min
    right.data = node.data[node.data[:,j_min] > t_min, :]
    right.box = m.copy(), M.copy()
    right.box[0][j_min] = t_min

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

In [5]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    v = np.prod(node.box[1] - node.box[0])
    node.response = n / (N * v)

# Decision Tree

In [6]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                perm = np.random.permutation(D)   # permute D indices
                left, right = make_decision_split_node(node, perm[:D_try]) #select :D_try of permuted indices
                stack += [left, right]
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response

In [7]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None

    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary:
        # Otherwise we would get thressholds which lie at the position of data points.
        data_unique = np.unique(node.data[:, j])
        # Compute candidate thresholds
        tj = (data_unique[:-1] + data_unique[1:]) / 2

        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            left_indices = node.data[:,j] <= t
            nl = np.sum(left_indices)
            ll = node.labels[left_indices]
            el = nl * (1 - np.sum(np.square(np.bincount(ll)/nl)))
            nr = n - nl
            lr = node.labels[node.data[:,j] > t]
            er = nr * (1 - np.sum(np.square(np.bincount(lr)/nr)))
            # choose the the best threshold that minimizes sum of Gini impurities
            if el + er < e_min:
                e_min = el + er
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()

    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min, :]
    left.labels = node.labels[node.data[:,j_min] <= t_min]
    right.data = node.data[node.data[:,j_min] > t_min, :]
    right.labels = node.labels[node.data[:,j_min] > t_min]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [8]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = node.labels.shape[0]
    node.response = np.bincount(node.labels, minlength=10) / node.N

In [9]:
def node_is_pure(node):
    '''
    check if 'node' contains only instances of the same digit
    '''
    return len(np.unique(node.labels)) == 1

# Evaluation of Density and Decision Tree

From here on, this is code from the sample solution.

In [10]:
# read and prepare the digits data
digits = load_digits()
data = digits["data"]
target = digits["target"]
# make data subsets for each digit
data_subsets = [data[target==i] for i in  range(10)]

Below we define a class for a generative classifier combining 10 instances of `DensityTree`. One instance of `DensityTree` is to be trained with only one class of data from the training data. Therefore training the generative classifier with full dataset will first separate the data into 10 subsets (each for one digit) and then trains 10 `DensityTree`, each trained for a digit subset. For prediction, a datapoint is be predicted by all 10 `DensityTree`s and each `DensityTree` returns $p(x|y)∗p(y)$  using `predict` of `DensityTree` class. The digit for the `DensityTree` maximizing  $p(x|y)∗p(y)$  is the prediction of the generative classifier.

A seperate class for a discriminative classifier is not used because it only requires one instance `DensityTree` trained with full dataset.

In [11]:
class GenerativeClassifier:
    def __init__(self):
        # create 10 instances of Densiry tree 
        self.trees = [DensityTree() for i in range(10)]
    
    def train(self, data, target, n_min=20):
        # train 10 trees, each with one subset of digits 
        data_subsets = [data[target==i] for i in  range(10)]
        N = len(target)
        for i, tree in enumerate(self.trees):
            tree.train(data_subsets[i], len(data_subsets[i]) / N, n_min)
            
    def predict(self, x):
        # return the digit for the DensityTree that maximizes p(x | y) * p(y)
        return np.argmax([tree.predict(x) for tree in self.trees])

Below we will train a `GenerativeClassifier` as discussed and a discriminative classifier with one `DecisionTree` with full dataset. And then plot the full training error confusion matrices.

### Evaluaton with `n_min = 20`

In [12]:
n_min = 20

# train with full dataset
# training generative classifier with DensityTrees
GC = GenerativeClassifier()
GC.train(data, target, n_min)
# training a discriminative classifier using one instance of DecisionTree
dt = DecisionTree()
dt.train(data, target, n_min)

# predict and compute the full training error confusion matrices
confusion_GC = np.zeros((10,10))
confusion_DC = np.zeros((10,10))
# for each digit subset
for i in range(10):
    # predict for with generative classifier
    predictions = np.array([GC.predict(j) for j in data_subsets[i]])
    confusion_GC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    # predict for with discriminative tive classifier
    predictions = np.array([np.argmax(dt.predict(j)) for j in data_subsets[i]])
    confusion_DC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100

print('Confusion Matrix for Generative Classifier using 10 instances of Density Tree')

print('-------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_GC, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)
print('Confusion Matrix for Discriminative Classifier using 1 instance of Decision Tree')

print('-------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_DC, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)

Confusion Matrix for Generative Classifier using 10 instances of Density Tree
-------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,59.89,2.75,0.00,6.04,3.85,0.00,2.20,25.27,0.00
2,0.00,4.52,64.41,1.69,0.00,0.56,0.00,0.00,28.81,0.00
3,0.00,0.55,2.73,62.84,0.00,0.00,0.00,2.73,30.60,0.55
4,0.00,0.00,0.00,0.00,80.11,1.10,0.00,17.13,0.00,1.66
5,0.00,1.10,0.00,7.69,0.00,75.82,0.00,4.95,10.44,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,1.12,1.68,0.00,94.97,2.23,0.00
8,0.00,12.07,0.00,0.00,0.57,6.32,0.00,3.45,77.59,0.00
9,0.00,4.44,0.56,22.22,1.67,3.89,0.00,5.00,13.89,48.33


Confusion Matrix for Discriminative Classifier using 1 instance of Decision Tree
-------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,94.38,0.00,0.56,0.00,1.69,0.00,1.12,0.56,0.00,1.69
1,0.00,90.11,3.85,2.20,0.55,0.55,0.00,0.00,1.10,1.65
2,0.00,1.13,88.70,3.39,0.00,0.00,0.56,0.00,4.52,1.69
3,0.00,2.19,2.73,84.70,0.00,0.55,0.00,1.09,0.00,8.74
4,1.66,4.42,0.55,1.66,86.74,1.10,2.21,0.00,1.10,0.55
5,0.00,2.20,0.55,2.75,0.55,83.52,0.55,0.00,0.55,9.34
6,0.00,0.55,1.10,0.00,2.76,0.00,94.48,0.00,1.10,0.00
7,1.68,0.00,0.00,4.47,0.56,3.91,0.00,86.03,1.12,2.23
8,1.15,3.45,6.32,2.87,1.72,0.00,0.00,1.15,82.18,1.15
9,0.56,2.78,1.67,1.11,0.00,0.56,0.00,0.56,1.11,91.67


In the confusion matrices above, the diagonals give the accuracies of predictions of digits, the off-diagonal elements are the error cases. You can see that the discriminative classifiers with decision trees always perform better than the generative classifiers with density trees, reflecting the fact that accurate generative modeling is harder than discriminative modeling.

Decreasing n_min increases the training accuracy of both classifiers, but this does not imply that the test accuracy would also increase!

Also if we repeat the experiments, the results can change significantly, because the success rate highly depends on the random selection of feature subsets when searching for the optimal split in each node.

# Density and Decision Forest

In [13]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            bootstrap_data = data[np.random.choice(len(data), size=len(data))]
            tree.train(bootstrap_data, prior, n_min)

    def predict(self, x):
        # compute the ensemble prediction
        return np.mean([tree.predict(x) for tree in self.trees])

In [14]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            bootstrap = np.random.choice(len(data), size=len(data))
            tree.train(data[bootstrap], labels[bootstrap], n_min)

    def predict(self, x):
        # compute the ensemble prediction
        return np.mean([tree.predict(x) for tree in self.trees], axis=0)

# Evaluation of Density and Decision Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
# Similarly as a generative classifier with 10 instances of DensityTrees, we define
#a generative classifier with 10 instances of DensityForest
class GenerativeClassifierDensityForest:
    def __init__(self, n_trees):
        # create 10 instances of DensiryForest
        self.forests = [DensityForest(n_trees) for i in range(10)]
    
    def train(self, data, target, n_min=20):
        # train 10 forests, each with one subset of digits 
        data_subsets = [data[target==i] for i in  range(10)]
        N = len(target)
        for i, forest in enumerate(self.forests):
            forest.train(data_subsets[i], len(data_subsets[i]) / N, n_min)
       
    def predict(self, x):
        # return the digit y that maximizes p(y | x)
        return np.argmax([forest.predict(x) for forest in self.forests])

Below we will train a `GenerativeClassifierDensityForest`, a discriminative classifier with one `DecisionForest` and sklearns `RandomForestClassifier`, each consisting of with 20 trees and with full dataset. And then plot the full training error confusion matrices. We evaluate for hyperparameter n_min = 20 and 10.

### Evaluation with `n_min = 20`

In [17]:
n_min = 20

# training generative classifier with DensityForests
GCF = GenerativeClassifierDensityForest(20)
GCF.train(data, target, n_min)

#traing a discriminative classifier using one instance of DecisionForest
df = DecisionForest(20)
df.train(data, target, n_min)

# traing  sklearn's predefined decision forest sklearn.ensemble.RandomForestClassifier
RFC = RandomForestClassifier(20, min_samples_split = n_min)
RFC.fit(data,target)

#data_subsets = [data[target==i] for i in  range(10)]
confusion_GC_Forest = np.zeros((10,10))
confusion_DC_Forest = np.zeros((10,10))
confusion_RFC  = np.zeros((10,10))
# for each digit subset make prediction and plot confusion matrix
for i in range(10):
    # predict with generative classifier 
    predictions = np.array([GCF.predict(j) for j in data_subsets[i]])
    confusion_GC_Forest[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    #predict with discriminative classifier
    predictions = np.array([np.argmax(df.predict(i)) for i in data_subsets[i]])
    confusion_DC_Forest[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100

    #predict for sklearn RF classifier
    predictions = RFC.predict(data_subsets[i])
    confusion_RFC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100

print('Confusion Matrix for Generative Classifier using 10 instances of Density Forest')

print('-----------------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_GC_Forest, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)
print('Confusion Matrix for Discriminative Classifier using 1 instance of Decision Forest')

print('-----------------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_DC_Forest, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)
print('Confusion Matrix for sklearn\'s RandomForest')

print('-----------------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_RFC, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)

Confusion Matrix for Generative Classifier using 10 instances of Density Forest
-----------------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,98.88,0.00,0.00,0.00,1.12,0.00,0.00,0.00,0.00,0.00
1,0.00,90.11,4.40,0.00,1.65,0.55,0.00,1.65,1.65,0.00
2,0.00,2.26,83.05,3.39,0.00,0.00,0.00,0.00,11.30,0.00
3,0.00,0.55,1.64,85.25,0.00,0.55,0.00,2.73,8.74,0.55
4,0.00,1.10,0.00,0.00,96.69,0.00,0.00,1.66,0.00,0.55
5,0.00,0.55,0.00,4.40,0.00,91.21,0.00,1.10,1.65,1.10
6,0.00,0.00,0.00,0.00,0.55,0.00,99.45,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,1.12,0.00,0.00,98.32,0.56,0.00
8,0.00,5.17,1.15,1.15,0.00,0.57,0.00,4.02,87.93,0.00
9,0.00,2.78,0.00,20.56,2.78,2.22,0.00,6.11,7.22,58.33


Confusion Matrix for Discriminative Classifier using 1 instance of Decision Forest
-----------------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00
9,0.00,0.00,0.00,1.11,0.00,0.00,0.00,0.00,0.00,98.89


Confusion Matrix for sklearn's RandomForest
-----------------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,99.44,0.00,0.00,0.00,0.56,0.00,0.00,0.00,0.00,0.00
1,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,98.36,0.00,0.00,0.00,1.09,0.55,0.00
4,0.00,0.00,0.00,0.00,98.90,0.00,0.00,1.10,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,99.45,0.55,0.00,0.00,0.00
6,0.00,0.55,0.00,0.00,0.00,0.00,99.45,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
8,0.00,1.15,0.00,0.00,0.00,0.57,0.00,0.00,97.70,0.57
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.67,0.56,97.78


### Evaluation with `n_min = 10`

In [18]:
n_min = 10

#training  a GenerativeClassifierDensityForest
GCF = GenerativeClassifierDensityForest(20)
GCF.train(data, target, n_min)

#training a discriminative classifier using one instance of DecisionForest
df = DecisionForest(20)
df.train(data, target, n_min)

# training  sklearn's predefined decision forest sklearn.ensemble.RandomForestClassifier
RFC = RandomForestClassifier(20, min_samples_split = n_min)
RFC.fit(data,target)

#data_subsets = [data[target==i] for i in  range(10)]
confusion_GC_Forest = np.zeros((10,10))
confusion_DC_Forest = np.zeros((10,10))
confusion_RFC  = np.zeros((10,10))
# for each digit subset make prediction and plot confusion matrix
for i in range(10):
    # predict with generative classifier 
    predictions = np.array([GCF.predict(j) for j in data_subsets[i]])
    confusion_GC_Forest[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100
    #predict with discriminative classifier
    predictions = np.array([np.argmax(df.predict(i)) for i in data_subsets[i]])
    confusion_DC_Forest[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100

    #predict for sklearn RF classifier
    predictions = RFC.predict(data_subsets[i])
    confusion_RFC[i,:] = np.bincount(predictions,minlength=10)/len(data_subsets[i])*100

print('Confusion Matrix for Generative Classifier using 10 instances of Density Forest')

print('-----------------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_GC_Forest, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)
print('Confusion Matrix for Discriminative Classifier using 1 instance of Decision Forest')

print('-------------------------------------------------------------------------------------')
display(
    pd.DataFrame(data = confusion_DC_Forest, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)
print('Confusion Matrix for sklearn\'s RandomForest')

print('--------------------------------------------')
display(
    pd.DataFrame(data = confusion_RFC, index =range(10), columns =range(10) )
    .rename_axis('Actual/Predicted', axis = 'columns')
)

Confusion Matrix for Generative Classifier using 10 instances of Density Forest
-----------------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,91.76,1.65,0.00,0.00,0.00,0.00,2.20,4.40,0.00
2,0.00,0.56,80.79,5.65,0.00,0.00,0.00,0.00,11.86,1.13
3,0.00,0.55,1.64,75.41,0.00,2.19,0.00,2.19,14.75,3.28
4,0.00,1.66,0.00,0.00,95.03,0.00,0.00,2.21,0.00,1.10
5,0.00,0.00,0.00,2.20,0.00,91.76,0.00,0.55,3.30,2.20
6,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,99.44,0.56,0.00
8,0.00,4.02,0.57,1.15,0.00,0.57,0.00,4.02,89.66,0.00
9,0.00,0.00,0.56,11.11,2.78,0.56,0.00,2.78,9.44,72.78


Confusion Matrix for Discriminative Classifier using 1 instance of Decision Forest
-------------------------------------------------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,99.45,0.00,0.00,0.00,0.55
6,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00


Confusion Matrix for sklearn's RandomForest
--------------------------------------------


Actual/Predicted,0,1,2,3,4,5,6,7,8,9
0,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,99.45,0.00,0.00,0.00,0.00,0.55
5,0.00,0.00,0.00,0.00,0.00,99.45,0.00,0.00,0.55,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00
9,0.00,0.00,0.00,0.56,0.00,0.00,0.00,0.00,0.00,99.44


We see that ensembles improve the performance of all methods in comparison to the single tree versions with same n_min.

Nonetheless, classification performance of simple generative models like the ones tested here is usually not competitive, unless we provide a lot more training data. Better generative models are therefore a very active field of research, and solutions based on deep neural networks, e.g. Generative Adversarial Networks (GANs), make much more efficient use of the available data and achieve impressive results.